# CZI
## Links
- [Dataset](https://datadryad.org/dataset/doi:10.5061/dryad.6wwpzgn2c)
- [Github](https://github.com/chanzuckerberg/software-mentions)

## Goal

Extract a basic column-wise dataset from the CZI dataset that has the following columns:

- `article_id`: the unique identifier for each article,
- `software_id`: the unique identifier for each software,
- `software_name`: the name of the software used in the article,
- `mention_type`: a classification of reason for mention (e.g., \"use\", \"create\", \"share\", etc.),
- `context`: the context of the mention (what was the surrounding text),
- `extra_fields`: what other fields are available in this dataset that we can use

## Three sub-datasets

### Raw
Raw, plain-text software mentions, as extracted by the NER model

```
license	location	pmcid	pmid	doi	pubdate	source	number	text	software	version	ID	curation_label
```
### Disambiguated
Disambiguated software mentions, after disambiguation

```
license	location	pmcid	pmid	doi	pubdate	source	number	text	software	version	ID	curation_label	mapped_to_software
```
### Linked 
Linked software mentions

```
ID	software_mention	mapped_to	source	platform	package_url	description	homepage_url	other_urls	license	github_repo	github_repo_license	exact_match	RRID	reference	scicrunch_synonyms
```

### What is available in this dataset based on the columns of somesci
- [x] `article_id`: Available; pmcid, pmid or doi
- [x] `software_id`: Available; RRID, Research Resource Identifier, but not many seem to have it (same as used for somisci?)
- [x] `software_name`: Available; raw disambiguated or linked. Using most processed level, linked
- [ ] `mention_type`: Not available 
- [x] `context`: Available
- [ ] `extra_fields`: added software_url based on 4 different url fields - replace limited RRID?

## Extracting basic columns

In [1]:
import polars as pl

In [2]:
ROOT_DATA_DIR = "../../data/CZI/"
RAW_PATH = ROOT_DATA_DIR + "raw/comm_raw.tsv.gz"
DISAMB_PATH = ROOT_DATA_DIR + "disambiguated/comm_disambiguated.tsv.gz"
LINKED_PATH = ROOT_DATA_DIR + "linked/metadata.tsv.gz"

In [3]:
disamb_df = pl.read_csv(DISAMB_PATH, separator="\t") #, infer_schema_length=100, n_rows=50000)
linked_df = pl.read_csv(LINKED_PATH, separator="\t") #, infer_schema_length=100, n_rows=50000)

### A look at the original data

In [4]:
disamb_df.describe()

statistic,license,location,pmcid,pmid,doi,pubdate,source,number,text,software,version,ID,curation_label,mapped_to_software
str,str,str,f64,f64,str,f64,str,f64,str,str,str,str,str,str
"""count""","""14770209""","""14770209""",1.4770209e7,1.4684219e7,"""14679097""",1.4770209e7,"""13406580""",1.4770209e7,"""14770209""","""14764379""","""1127612""","""14770209""","""14770209""","""14770207"""
"""null_count""","""0""","""0""",0.0,85990.0,"""91112""",0.0,"""1363629""",0.0,"""0""","""5830""","""13642597""","""0""","""0""","""2"""
"""mean""",null,null,5.9107e6,2.9085e7,null,2017.106828,null,29.593835,null,null,null,null,null,null
"""std""",null,null,1.7456e6,4.3322e6,null,3.607958,null,109.923538,null,null,null,null,null,null
"""min""","""comm""","""comm/20_Century_Br_Hist/PMC480…",176545.0,1.777407e6,""" 10.1186/1477-5956-10-26""",1797.0,"""""""""""""""""Administration""""""""""""""""",0.0,""" # 198 genes mapped to this te…",""" MGA""","""#20""","""SM0""","""not_curated""","""#GenomicDay"""
"""25%""",null,null,4.529189e6,2.6161174e7,null,2015.0,null,8.0,null,null,null,null,null,null
"""50%""",null,null,6.128579e6,3.0105754e7,null,2018.0,null,18.0,null,null,null,null,null,null
"""75%""",null,null,7.431423e6,3.2730277e7,null,2020.0,null,35.0,null,null,null,null,null,null
"""max""","""comm""","""comm/psychopraxis/PMC8325535.n…",8.51084e6,3.4637085e7,"""10.9745/GHSP-D-21-00233""",2022.0,"""𝜑XANES analysis""",20116.0,"""𝜀c regressions and comparisons…","""鼠源及人源化BCMA CAR-T的转染效率""","""应用SPSS22.0软件进行统计学分析""","""SM999999""","""unclear""","""∗BEAST"""


In [18]:
disamb_df

license,location,pmcid,pmid,doi,pubdate,source,number,text,software,version,ID,curation_label,mapped_to_software
str,str,i64,f64,str,i64,str,i64,str,str,str,str,str,str
"""comm""","""comm/Micropl/PMC8475362.nxml""",8475362,null,"""10.1186/s43591-021-00017-9""",2021,"""Particle selection and identif…",7,"""Then, all items were photograp…","""Olympus CellSens""",null,"""SM0""","""not_curated""","""Olympus cellSens"""
"""comm""","""comm/Micropl/PMC8475362.nxml""",8475362,null,"""10.1186/s43591-021-00017-9""",2021,"""Particle selection and identif…",8,"""Spectra were then vector norma…","""OPUS""",null,"""SM1""","""unclear""","""OPUS"""
"""comm""","""comm/Micropl/PMC8475362.nxml""",8475362,null,"""10.1186/s43591-021-00017-9""",2021,"""Statistical analysis""",12,"""Model fit was assessed through…","""R package DHARMa""",null,"""SM2""","""not_curated""","""DHARMa"""
"""comm""","""comm/Micropl/PMC8475362.nxml""",8475362,null,"""10.1186/s43591-021-00017-9""",2021,"""Statistical analysis""",12,"""Analyses and plotting were per…","""R""",null,"""SM3""","""software""","""R"""
"""comm""","""comm/Micropl/PMC8475362.nxml""",8475362,null,"""10.1186/s43591-021-00017-9""",2021,"""Statistical analysis""",12,"""Analyses and plotting were per…","""ggplot2""",null,"""SM4""","""software""","""ggplot2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""comm""","""comm/Nutrients/PMC6683272.nxml""",6683272,3.1248006e7,"""10.3390/nu11071443""",2019,"""2.2. Outcome Measures""",6,"""All data were obtained by revi…","""MetaVision""",null,"""SM53566""","""not_curated""","""MetaVision"""
"""comm""","""comm/Nutrients/PMC6683272.nxml""",6683272,3.1248006e7,"""10.3390/nu11071443""",2019,"""2.2. Outcome Measures""",6,"""All data were obtained by revi…",null,"""5.46.42""","""SM4442""","""not_curated""","""not_disambiguated"""
"""comm""","""comm/Nutrients/PMC6683272.nxml""",6683272,3.1248006e7,"""10.3390/nu11071443""",2019,"""2.2. Outcome Measures""",6,"""All data were obtained by revi…","""iMDsoft""","""5.46.42""","""SM53019""","""not_curated""","""iMDsoft"""


In [5]:
linked_df.describe()

statistic,ID,software_mention,mapped_to,source,platform,package_url,description,homepage_url,other_urls,license,github_repo,github_repo_license,exact_match,RRID,reference,scicrunch_synonyms
str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str
"""count""","""149015""","""149015""","""149015""","""149015""","""17540""","""149015""","""116070""","""36306""","""18766""","""13485""","""143835""","""39464""",149015.0,"""18766""","""22134""","""18766"""
"""null_count""","""0""","""0""","""0""","""0""","""131475""","""0""","""32945""","""112709""","""130249""","""135530""","""5180""","""109551""",0.0,"""130249""","""126881""","""130249"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,0.903902,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""SM100000""","""'O""",""" AnaMorph""","""Bioconductor Index""","""Bioconductor""","""https://cran.r-project.org/web…",""" Multilevel Modeling in Epide…","""[""http://www.maths.soton.ac.uk…","""['Mutation', 'Surveyor', 'soft…","""ACM""","""<https://github.com/zhangjunpe…","""0BSD""",0.0,"""SCR_000004""","""<pre> @Article{, author …","""[""a character of the italian c…"
"""25%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""max""","""SM999993""","""ÖGD""","""zzip""","""SciCrunch API""","""Pypi""","""https://www.bioconductor.org/p…","""🪱 PARASITE || A parallel sente…","""[]""","""[]""","""file LICENSE""","""https://github.com/zzzzbw/Fame""","""Zlib""",1.0,"""SCR_021924""","""https://doi.org/doi:10.18129/B…","""['zymo research corporation', …"


### Exclude the mentions marked as not_software:


In [6]:
disamb_df_clean = disamb_df.filter(pl.col("curation_label") != "not_software")
# ratio of clean to not clean data
disamb_df_clean.shape[0] / disamb_df.shape[0]


0.9012042415919774

### Select and rename columns

In [12]:
# Select core columns from the disambigued data
core_df = disamb_df_clean.select([
    pl.col("doi").alias("article_doi"), # Using DOI as the main ID
    pl.col("pmcid").alias("article_pmcid"), # Adding pmcid to enable comparison with somesci
    pl.col("ID").alias("software_id_CZI"), # The dataset specific ID
    pl.col("software").alias("software_name"),
    pl.col("text").alias("context"),
    # pl.col(" ").alias("mention_type") # Mention type does not exist, and extracting from context might not be reliable enough
]) 

# Software IDs from linked data
software_info_df = linked_df.select([
    pl.col("ID").alias("software_id_CZI"),# The dataset specific ID
    pl.col("RRID").alias("software_rrid"), # Disambiguated software identifier (if available)

    # combine the different urls into one called software_url - list of unique not nulls:
    # TODO: these are all currently str "lists" except for package_url, need to fix that later
    (
    pl.concat_list([
        pl.col("homepage_url"),
        pl.col("other_urls"),
        pl.col("github_repo"),
        pl.col("package_url")  # this is originally a string
    ]).alias("software_url")
    )

])


# Join  - add software ids to the entries in the core df. joining left on dataset specific ID
merged_df = core_df.join(software_info_df, on="software_id_CZI", how="left")

# Rearrange so its the same as the somisci
# Although this also contains the contexta and does not contain mention type as that is not available
final_df = merged_df.select([
    "article_doi",
    "article_pmcid",
    "software_rrid",
    "software_id_CZI",
    "software_name",
    "software_url",
    "context",
])

final_df



article_doi,software_id_CZI,software_rrid,software_name,software_url,context,article_pmcid
str,str,str,str,list[str],str,i64
"""10.1186/s43591-021-00017-9""","""SM0""",null,"""Olympus CellSens""",null,"""Then, all items were photograp…",8475362
"""10.1186/s43591-021-00017-9""","""SM1""",null,"""OPUS""",null,"""Spectra were then vector norma…",8475362
"""10.1186/s43591-021-00017-9""","""SM2""",null,"""R package DHARMa""",null,"""Model fit was assessed through…",8475362
"""10.1186/s43591-021-00017-9""","""SM3""",null,"""R""","[null, null, … ""https://github.com/ncornwell/R""]","""Analyses and plotting were per…",8475362
"""10.1186/s43591-021-00017-9""","""SM3""",null,"""R""","[null, null, … ""https://github.com/dmpe/R""]","""Analyses and plotting were per…",8475362
…,…,…,…,…,…,…
"""10.3390/nu11071443""","""SM53566""",null,"""MetaVision""",null,"""All data were obtained by revi…",6683272
"""10.3390/nu11071443""","""SM4442""",null,null,null,"""All data were obtained by revi…",6683272
"""10.3390/nu11071443""","""SM53019""",null,"""iMDsoft""",null,"""All data were obtained by revi…",6683272


### Save dataframe

In [17]:
# Store to parquet
final_df.write_parquet(
    ROOT_DATA_DIR + "processed-CZI-dataset.parquet",
)

### Stats

In [13]:
final_df.describe()

statistic,article_doi,software_id_CZI,software_rrid,software_name,software_url,context,article_pmcid
str,str,str,str,str,f64,str,f64
"""count""","""16056698""","""16158993""","""4223665""","""16153163""",9.46531e6,"""16158993""",1.6158993e7
"""null_count""","""102295""","""0""","""11935328""","""5830""",6.693683e6,"""0""",0.0
"""mean""",null,null,null,null,null,null,5.9125e6
"""std""",null,null,null,null,null,null,1.7347e6
"""min""",""" 10.1186/1477-5956-10-26""","""SM0""","""SCR_000004""",""" MGA""",null,""" # 198 genes mapped to this te…",176545.0
"""25%""",null,null,null,null,null,null,4.540975e6
"""50%""",null,null,null,null,null,null,6.126143e6
"""75%""",null,null,null,null,null,null,7.415876e6
"""max""","""10.9745/GHSP-D-21-00144""","""SM999999""","""SCR_021924""","""鼠源及人源化BCMA CAR-T的转染效率""",null,"""𝜀c regressions and comparisons…",8.51084e6


In [15]:
# not many RRIDs
null_ratio = final_df.select(
    (pl.col("software_rrid").is_null().sum() / pl.len())
).item()

print(null_ratio)


0.7386183037519727
